# German Lawyer 

A Jupyter notebook to help navigate the residency law in Germany. This project uses local embeddings and models to do RAG (Retreival Augmented Generation) over the German residency law. This means that the model is living locally on the computer, the embeddings are done locally, and the querying is done locally.

You can ask questions like:

* What are the requirements for a Blue Card?
* What are the requirements for a student visa?
* What are the requirements for a work visa?

I've taken the Aufenthaltsgesetz and Aufenthaltsverordnung from Gesetze im Internet as XML and using the Unstructured XML loader, I've loaded them in as a LangChain document.

## Project Steps

1. Load the XML files into a LangChain document
2. Split the document into sections
3. Embeddings
4. Vector Store
5. LLM Setup (Prompt Template & Querying)

### 1: Use LangChain Unstructured XML Loader to Load in the German Residence Law

In [72]:
from langchain.document_loaders import UnstructuredXMLLoader, TextLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

from langchain.embeddings import OllamaEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma, Qdrant

from langchain.llms import Ollama

from langchain.chains import RetrievalQA   
from langchain.prompts import PromptTemplate 

import time

from langchain.chains import ConversationalRetrievalChain


### 2: Load & Split the Text

In [96]:
# German Residence Law
# source: https://www.gesetze-im-internet.de/aufenthv/BJNR294510004.html
file = "german-law/laws/Aufenthaltsverordnung/BJNR294510004.xml"

# load German Residence Law XML file with UnstructuredXMLLoader
loader = UnstructuredXMLLoader(file_path = file)
docs = loader.load()

**Recursive Character Text Splitter**

Use recursive character text splitter to split texts into chunks of 1000

In [46]:
# Try with the RecursiveCharacterTextSplitter

r_text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap  = 150)
r_texts = r_text_splitter.create_documents([docs[0].page_content])


In [45]:
# Try with the CharacterTextSplitter

c_text_splitter = CharacterTextSplitter(chunk_size = 1500, chunk_overlap  = 150)
c_texts = c_text_splitter.create_documents([docs[0].page_content])


Created a chunk of size 7988, which is longer than the specified 1500
Created a chunk of size 3185, which is longer than the specified 1500


### 3: Create Vectorstore

In [54]:
# OpenAI Embeddings, Chroma as vectorstore
openai_vectorstore = Chroma.from_documents(documents = r_texts, embedding=OpenAIEmbeddings())
retreiver = openai_vectorstore.as_retriever()

In [97]:
# Ollama Embeddings (openhermes2.5), Qdrant as vectorstore 
# Note: (Chroma does not work, as Ollama creates 4096-dimensional vectors and Chroma accepts 1536-dimensional vectors only)

# loader = TextLoader("/Users/ingrid/Developer/GitHub/lawyer/README.md")
# docs = loader.load()

test_text_splitter = CharacterTextSplitter(chunk_size = 1500, chunk_overlap  = 150)
test_texts = test_text_splitter.create_documents([docs[0].page_content])


ollama_vectorstore = Qdrant.from_documents(
    documents=test_texts, 
    embedding=OllamaEmbeddings(
        model="openhermes2.5-mistral:7b-q5_K_M",
        show_progress=True,
        ),
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="texts",
)


Created a chunk of size 7988, which is longer than the specified 1500
Created a chunk of size 3185, which is longer than the specified 1500
OllamaEmbeddings: 100%|██████████| 63/63 [02:54<00:00,  2.77s/it]


In [31]:
ollama_retreiver = ollama_vectorstore.as_retriever()

### 4: LLM Setup

**LLM Setup**

In [48]:
# Temporarily set the model to 'mistral'
llm = Ollama(model='openhermes2.5-mistral:7b-q5_K_M')

**Retrieval QA Prompt**

#### Let the Not a Lawyer be a Not Lawyer

In [92]:
# define a function which takes as inputs the llm, embeddings, and outputs the result (printed)
# ideally log as tags which llm and embeddings was used, allow me to categorize outputs as (good, not good, or comment in some ways)
import time 
def test_llm(vectorstore, model, question):

    start = time.time()

    # build prompt 
    template = """
        You are a professional, courteous, helpful AI legal assistant for question-answering tasks about residency law for people living in, or considering moving to Germany. 
        Use the following pieces of retrieved context from the German Law (delimited in $$$ $$$)to answer the question. Always cite the source of your answer.
        If you don't know the answer, just say that you don't know. Do not make anything up!
        Always cite the source of your answer! And, don't forget to empathize with the user - they are probably stressed out and need help!
        Question: {question} 

        Context: $$$ {context} $$$

        Answer:

        """

    # create prompt template
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    # set qa chain
    qa_chain_mr = RetrievalQA.from_chain_type(
        Ollama(model=model), 
        retriever = vectorstore.as_retriever(),
        chain_type="stuff", 
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )

    # get the result
    result = qa_chain_mr({"query": question})

    # print the result
    print(result["result"])

    end = time.time()
    elapsed_time = end - start
    print("The function took", elapsed_time, "seconds to run.")


In [93]:
frage = "How can I move to germany? I'm from the United states."
test_llm(openai_vectorstore, 'openhermes2.5-mistral:7b-q5_K_M', frage)




To move to Germany from the United States, you will need a residence permit (Aufenthaltstitel). There are different types of residence permits depending on your situation. If you have a family member who is a German citizen and lives in Germany, you can apply for a family reunification residence permit (Familienzusammenführung Aufenthaltstitel).

You will also need a valid visa to enter Germany, such as a visitor's visa (Schengen Visa), which allows you to stay for up to 90 days. After that, you can apply for a residence permit from within Germany. You may also be eligible for a national visa if you have a job offer or plan to study in Germany.

The process of obtaining a residence permit and visa can be complex, so it's recommended to consult with an immigration attorney or the German embassy for more specific information and guidance tailored to your situation.
The function took 29.580472946166992 seconds to run.


In [98]:
frage = "I just got a job in Germany paying me 80,000 euros annually. What are my options for a residence permit?"
test_llm(ollama_vectorstore, 'mistral', frage)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.54it/s]



        The German Foreigners Residence Act regulates the residence of foreigners in Germany and provides for the issuance of various types of residence permits. The Act divides into several sections, each of which sets out specific rules and requirements for a particular type of residence permit. 

        Section 1 deals with the general provisions of the Act, including the definitions of key terms such as "foreigner" and "residence permit." 

        Section 2 outlines the different types of residence permits that may be issued, including temporary and permanent residence permits for various purposes such as study, work, and family reunification. 

        Section 3 sets out the requirements for each type of residence permit, including the duration of validity, the purpose of the visit, and any other conditions or restrictions that must be met. 

        Section 4 deals with the issuance of residence permits for specific groups of foreigners, such as students and researchers. 

   

In [80]:
frage = "What are the requirements for a Blue Card?"
test_llm(openai_vectorstore, 'llama2', frage)

 The requirements for a Blue Card in Germany are as follows:

* The applicant must be a non-EU citizen who has been residing in Germany for at least six months on a valid residence permit.
* The applicant must have a legitimate reason for obtaining the Blue Card, such as employment, self-employment, study, or family reunification.
* The applicant must meet the income requirements, which are based on the Federal Statistical Office's (Destatis) minimum salary rates for the respective occupation and location.
* The applicant must have health insurance coverage in Germany.
* The applicant must pass a language proficiency test in German or English, depending on the job requirements.
* The applicant must provide proof of adequate accommodation in Germany.

It is important to note that the requirements for a Blue Card may vary depending on the individual's country of origin and the purpose of their stay in Germany. Therefore, it is recommended to consult the German Federal Employment Agency (

In [69]:
frage = "What are the requirements for a Blue Card?"
test_llm(openai_vectorstore, 'openhermes2.5-mistral:7b-q5_K_M', frage)

No, I don't know the answer. Please provide more context or clarify the question.
The function took 17.338717937469482 seconds to run.


In [82]:
frage = "What are the requirements for a Blue Card?"
test_llm(openai_vectorstore, 'mistral', frage)


To answer the question "What are the requirements for a Blue Card?", we need to find the specific information that outlines the necessary conditions for receiving a Blue Card in Germany. According to the provided context, there are several scenarios where a person may be eligible for a Blue Card. Here are the relevant pieces of information:

1. The person must have been in possession of a valid Blue Card from another EU member state for at least six months before applying for a new one.
2. If the person already had a Blue Card from an EU member state prior to their current application, it must have been issued by a different member state than the one they are currently applying to.
3. The person's family members who are in possession of an Aufenthaltstitel zum Familiennachzug and are being hosted by the same state as their Blue Card must also meet these requirements.
4. The application for a new Blue Card or Aufenthaltserlaubnis zum Familiennachzug must be submitted within one month o

In [99]:
frage = "What are the requirements for a Blue Card?"
test_llm(ollama_vectorstore, 'mistral', frage)

OllamaEmbeddings: 100%|██████████| 1/1 [00:05<00:00,  5.14s/it]


        
        Translation:
        
       According to the Federal Residence Act (AufenthG), foreigners who have obtained a residence permit for a longer than 90-day stay in Germany or wish to enter into an employment relationship with the federal government, must provide proof of their right of residence. For this purpose, the following information is required:
       4.a) The mark of the residence visa, b) The date of issuance, c) The category of the residence permit, d) Its validity period, e) The number of days allowed for stay, f) The area of application of the residence permit and the number of authorized entries into that area, and g) Any conditions, restrictions, and other requirements associated with it.
      
        4.a) Date of decision, b) Reasons for decision.
       
       Additionally, foreigners who have obtained a German pass replacement (AufenthV § 3), who have been granted permission by agreement between states to use non-German official documents as a substit

In [100]:
frage = "What are the requirements for a Blue Card?"
test_llm(ollama_vectorstore, 'llama2', frage)

OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it]


 Thank you for reaching out to me! I'm happy to help you with your question about the requirements for a Blue Card.

To answer your question, according to Section 4 of the German Residence Act (Aufenthaltsgesetz - AufenthV), a Blue Card is required for non-EU citizens who wish to reside in Germany for more than 90 days. The requirements for a Blue Card are as follows:

1. The applicant must be a non-EU citizen.
2. The applicant must have a valid travel document (passport or identity card).
3. The applicant must have a negative result from a drug test.
4. The applicant must have health insurance that covers them in Germany.
5. The applicant must have enough funds to support themselves during their stay in Germany.
6. The applicant must meet the requirements for admission to the Federal Republic of Germany, as laid down in Section 16b of the German Residence Act.
7. The applicant must meet the requirements for a residence title, as laid down in Section 19 of the German Residence Act.
8. 

In [101]:
frage = "What are the requirements for a Blue Card?"
test_llm(ollama_vectorstore, 'openhermes2.5-mistral:7b-q5_K_M', frage)

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.37s/it]



To apply for a Blue Card in Germany, the requirements are as follows:

1. You must have a valid passport or other acceptable travel document.
2. You should have a job offer from a German employer and have a recognized higher education degree or at least 5 years of work experience in your profession.
3. You need to submit an application for a residency permit (EU Blue Card) at the local foreigners' registration office (Ausländerbehörde).
4. You must provide evidence of sufficient health insurance coverage in Germany.
5. You have to prove that you have adequate financial resources or income to support yourself and any dependents during your stay in Germany.
6. You need to submit biometric data, such as fingerprints, and a photograph.
7. Dependents (spouse and children) must also meet specific criteria to be eligible for a Blue Card family member visa. 
8. Finally, you have to pay the application fee.

Please refer to the German Law context provided above for more detailed information on

In [ ]:
frage = "How can a resident of Germany obtain citizenship?"
test_llm(ollama_vectorstore, 'mistral', frage)

In [ ]:
test_llm(ollama_vectorstore, 'llama2', frage)

### Findings:

Recursive Text Splitter
 * mistral: 19.5s
 * llama2: 26.2s

 Text splitter
 * mistral: 26.5s
 * llama2: 79.7s

 Conclusion: mistral is faster, recursive character text splitter is faster. Why? No idea.

### Set up memory

In [75]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

**Define a function to run the conversational retrieval chain (including memory)**

In [76]:
def test_llm_inkl_memory(vectorstore, model, question):

    retriever=vectorstore.as_retriever()
    qa = ConversationalRetrievalChain.from_llm(
        Ollama(model=model),
        retriever=retriever,
        memory=memory
    )
    result = qa({"question": question}) 
    print(result['answer'])

In [77]:
# Get the answer
question = "How do I get a bluecard?"
test_llm_inkl_memory(openai_vectorstore, 'mistral', question)


To apply for a Blue Card EU, you must first have a valid Blue Card EU from another member state of the European Union. You must also have been in possession of a valid Blue Card EU from another member state of the European Union for at least six months immediately before applying for the new Blue Card EU. If you are applying for a Blue Card EU for your family members, they must also be in possession of valid Blue Cards EU from the same member state as you.

To apply for a Blue Card EU and an Aufenthaltserlaubnis zum Familiennachzug (family reunification permit), you must do so within one month of entering the country. You must also meet the required conditions set forth in § 30a of the Aufenthaltsgesetz (residence law) for Wiedereinreise in das Bundesgebiet.

If you wish to extend your ICT-Karte (intra-corporate transferee card), you must submit an application within one month of entering the country and meet the required conditions set forth in § 19 of the Aufenthaltsgesetz (residenc

In [78]:

question = "I don't already have a bluecard, but I just got a job offer for 100k. Can I get a bluecard?"
test_llm_inkl_memory(openai_vectorstore, 'mistral', question)

No, you cannot obtain a Blue Card EU without having a valid one from another member state of the European Union. According to the given context, if an individual has a Blue Card EU that was issued by a member state of the European Union and is about to expire, they can apply for a new Blue Card EU without leaving the country as long as they meet the requirements for a new one. However, if an individual does not have a valid Blue Card EU from another member state of the European Union, they will need to obtain a new Blue Card EU by going through the process of being granted a new one from a different member state.


In [79]:

question = "How do i get one if i haven't had one before?"
test_llm_inkl_memory(openai_vectorstore, 'mistral', question)


To obtain a Blue Card EU from another member state of the European Union, there are certain requirements that must be met. Firstly, the applicant must have possessed a Blue Card EU from a member state of the European Union for at least six months prior to applying for the new Blue Card EU. Secondly, the applicant must not be from the same member state as the one issuing the new Blue Card EU. Thirdly, the applicant must have an active employment relationship in Germany and must have a valid work permit.

In addition to these requirements, there are certain time limits for applying for a Blue Card EU. The application must be made within one month of entering Germany and the applicant must provide all necessary documentation within that time frame. If the applicant is unable to meet this deadline, they may need to extend their stay in Germany until they can submit a complete application.

Overall, obtaining a Blue Card EU from another member state of the European Union typically takes se